# Visualização da Informação
## Escola de Matemática Aplicada - Fundação Getúlio Vargas
## Mestrado em Modelagem Matemática

Aluno: Gianlucca Devigili<br>
Github do projeto: https://github.com/GDevigili/information-visualization-homeworks

## Trabalho 2: Análise e reprodução de uma visualização reconhecida competente ou relevante historicamente

+ Parte1: Encontrar os dados (compartilhar referências de dados no slack)
+ Parte2: Fazer uma análise de qual seria a função pretendida com a visualização proposta. 
+ Parte3: Fazer uma reprodução da visualização escolhida utilizando uma ferramenta computacional atual(de preferência a mesma escolhida por vocês no trabalho 1)
+ Parte4: Propor alguma modificação (fundamentando conceitualmente) na visualização proposta. Exemplo: Incluir anotação, incluir interatividade, modificar título ou legenda, adicionar informação, etc.

In [7]:
import pandas as pd
import altair as alt

# scrapping
from bs4 import BeautifulSoup as bs
import requests
import vega_datasets

### Parte 1: Encontrar os dados

Tive certa dificuldade de encontrar o dataframe em `.csv`, portanto fiz um scrapping simples da tabela encontrada em <https://medium.com/@zumaia/how-florence-nightingales-diagram-saved-millions-of-people-from-fatal-epidemic-disease-dfad7348b05d>.

In [9]:
html = requests.get("http://understandinguncertainty.org/node/214.")
soup = bs(html.text)

# seleciona a tag table
table = soup.find('table')

# extrai o nome das colunas da tabela
column_names = []
for name in table.find_all('tr')[1]:
    if name != "\n":
        column_names.append(name.text)

for i in range(2, 5):
    column_names[i] += '(Deaths)'
    column_names[i+3] += '(Annual rate per 1000)'
        
# extrai as linhas da tabela
trows = soup.find('tbody').find_all('tr')[2:]
tcols = []

# itera sobre as linhas
for tr in soup.find('tbody').find_all('tr')[2:]:
    # pega os valores da lista de 'td' e remove ' ' e '\n'
    cols = [val.text.replace('\n', '').replace(' ', '').replace('_', '') for val in tr.find_all('td')]
    # adiciona a lista de colunas
    tcols.append(cols)

# cria o dataset 
column_names[0] = 'Date'
df_deaths = pd.DataFrame(tcols, columns = column_names)

'''
# cria um dataset para as datas
dict_months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
month_year = []

for date in df_deaths['Date']:
    # cria uma lista de listas do tipo [[mes, ano], [mes, ano], [mes, ano], ...]
    month_year.append([int(dict_months[date[0:3]]), int(date[3:])])
    
df_date = pd.DataFrame(month_year, columns=['Month', 'Year'])
'''
# converte para int64 e float64 as colunas numéricas
for col in df_deaths.columns[1:]:
    df_deaths[col] = df_deaths[col].apply(pd.to_numeric)
    
df_deaths['Date'] = vega_datasets.data.crimea()['date']

df_deaths

,Date,Average size of army,Zymotic diseases(Deaths),Wounds & injuries(Deaths),All other causes(Deaths),Zymotic diseases(Annual rate per 1000),Wounds & injuries(Annual rate per 1000),All other causes(Annual rate per 1000)
0,1854-04-01,8571,1,0,5,1.4,0.0,7.0
1,1854-05-01,23333,12,0,9,6.2,0.0,4.6
2,1854-06-01,28333,11,0,6,4.7,0.0,2.5
3,1854-07-01,28722,359,0,23,150.0,0.0,9.6
4,1854-08-01,30246,828,1,30,328.5,0.4,11.9
5,1854-09-01,30290,788,81,70,312.2,32.1,27.7
6,1854-10-01,30643,503,132,128,197.0,51.7,50.1
7,1854-11-01,29736,844,287,106,340.6,115.8,42.8
8,1854-12-01,32779,1725,114,131,631.5,41.7,48.0
9,1855-01-01,32393,2761,83,324,1022.8,30.7,120.0


,date,wounds,other,disease
0,1854-04-01,0,110,110
1,1854-05-01,0,95,105
2,1854-06-01,0,40,95
3,1854-07-01,0,140,520
4,1854-08-01,20,150,800
5,1854-09-01,220,230,740
6,1854-10-01,305,310,600
7,1854-11-01,480,290,820
8,1854-12-01,295,310,1100
9,1855-01-01,230,460,1440
